<a href="https://colab.research.google.com/github/sezinbiner/Covid_X_ray_Classification/blob/main/traditional_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_dir = "/content/gdrive/MyDrive/bitirme dataset/train"
test_dir = "/content/gdrive/MyDrive/bitirme dataset/test"
validation_dir = "/content/gdrive/MyDrive/bitirme dataset/validation"
class_names = ['covid', 'normal']
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),  
        batch_size=20,
        shuffle=True,
        class_mode='binary')
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        shuffle=True,
        class_mode='binary')
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=1,
        class_mode='binary')

Found 4800 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 1232 images belonging to 2 classes.


In [ ]:
def extract_features(number_of_images, generator, batch_size):
  features = np.zeros(shape=(number_of_images, extraction_layer.output_shape[1], extraction_layer.output_shape[2], extraction_layer.output_shape[3]))
  labels = np.zeros(shape=(number_of_images))
  i = 0
  print("loop")
  for inputs_batch, labels_batch in generator:
      #print(i)
      features_batch = extraction_layer.predict(inputs_batch)
      features[i * batch_size : (i + 1) * batch_size] = features_batch
      labels[i * batch_size : (i + 1) * batch_size] = labels_batch
      i += 1
      if i * batch_size >= number_of_images:
          break
  print("loop over")
  return features, labels

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
import numpy as np

extraction_layer = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)
extraction_layer.summary()

58900480/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
train_features, train_labels = extract_features(4800, train_generator, 20)
test_features, test_labels = extract_features(1232, test_generator, 1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import sklearn

In [ ]:
def knn(x_train,y_train,x_test,y_test):
    rand_x = []
    rand_test = []
    for img in x_train:
      t=np.ravel(img)
      rand_x.append(t)
    for img in x_test:
      t=np.ravel(img)
      rand_test.append(t)
    pixels = pd.DataFrame(rand_x)
    labels = pd.DataFrame(y_train)
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=2,
                     weights='distance')

    knn.fit(rand_x,y_train)
    y_pred=knn.predict(rand_test)
    from sklearn import metrics
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Test data metrics:")
    print(sklearn.metrics.classification_report(y_true= y_test, y_pred= y_pred))

    return metrics.accuracy_score(y_test, y_pred)

In [ ]:
knn(train_features,train_labels,test_features,test_labels)

Accuracy: 0.9512987012987013
Test data metrics:
              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95       616
         1.0       0.95      0.95      0.95       616

    accuracy                           0.95      1232
   macro avg       0.95      0.95      0.95      1232
weighted avg       0.95      0.95      0.95      1232



0.9512987012987013

In [ ]:
def SVM(x_train,y_train,x_test,y_test):
    
    rand_x = []
    rand_test = []
    for img in x_train:
      t=np.ravel(img)
      rand_x.append(t)
    for img in x_test:
      t=np.ravel(img)
      rand_test.append(t)
    pixels = pd.DataFrame(rand_x)
    labels = pd.DataFrame(y_train)
    from sklearn.svm import SVC
    svc=SVC(kernel='rbf')
    svc.fit(rand_x,y_train)
    y_pred=svc.predict(rand_test)
    from sklearn import metrics
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Test data metrics:")
    print(sklearn.metrics.classification_report(y_true= y_test, y_pred= y_pred))

    return metrics.accuracy_score(y_test, y_pred)


In [ ]:
SVM(train_features,train_labels,test_features,test_labels)

Accuracy: 0.9448051948051948
Test data metrics:
              precision    recall  f1-score   support

         0.0       0.95      0.94      0.94       616
         1.0       0.94      0.95      0.95       616

    accuracy                           0.94      1232
   macro avg       0.94      0.94      0.94      1232
weighted avg       0.94      0.94      0.94      1232



0.9448051948051948

In [ ]:
def random_forest(x_train,y_train,x_test,y_test):
    
    rand_x = []
    rand_test = []
    for img in x_train:
      t=np.ravel(img)
      rand_x.append(t)
    for img in x_test:
      t=np.ravel(img)
      rand_test.append(t)
    pixels = pd.DataFrame(rand_x)
    labels = pd.DataFrame(y_train)
    from sklearn.ensemble import RandomForestClassifier
    rf=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)
    rf.fit(rand_x,y_train)
    y_pred=rf.predict(rand_test)
    from sklearn import metrics
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    print("Test data metrics:")
    print(sklearn.metrics.classification_report(y_true= y_test, y_pred= y_pred))

    return metrics.accuracy_score(y_test, y_pred)

In [ ]:
random_forest(train_features,train_labels,test_features,test_labels)

Accuracy: 0.939935064935065
Test data metrics:
              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94       616
         1.0       0.94      0.94      0.94       616

    accuracy                           0.94      1232
   macro avg       0.94      0.94      0.94      1232
weighted avg       0.94      0.94      0.94      1232



0.939935064935065